# Customing Models and Training Algorithms

### Custom Loss Functions
Below we will see a custom implementation of Huber loss to compute each instance's loss.

Let's start by loading and preparing the California housing dataset. We first load it, then split it into a training set, a validation set and a test set, and finally we scale it:

In [5]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

X_train.shape, X_test.shape, X_valid.shape

((11610, 8), (5160, 8), (3870, 8))

In [1]:
def huber_fn(y_true, y_pred):
    error = y_true, y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [6]:
input_shape = X_train.shape[1:]

input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [7]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

history = model.fit(X_train_scaled, y_train, epochs=2, validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 3ms/step - loss: 0.8433 - mae: 2.0871 - val_loss: 0.8138 - val_mae: 2.1075
Epoch 2/2
363/363 [==============================] - 1s 1ms/step - loss: 0.7984 - mae: 2.0835 - val_loss: 0.8003 - val_mae: 2.0815


### Saving / Loading Models with Custom Objects

In [8]:
model.save("my_model_with_a_custom_loss.h5")

To load a model containing custom loss function, you'll need to provide a dictionary that maps the function name to the actual function.

In [9]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [10]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.7969 - mean_absolute_error: 2.0830 - val_loss: 0.7887 - val_mean_absolute_error: 2.0661
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.7962 - mean_absolute_error: 2.0832 - val_loss: 0.7830 - val_mean_absolute_error: 2.0487


The above huber loss considers a threshold of -1 to +1. For a configured loss function:

In [11]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true, y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [12]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])

In [13]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.1825 - mae: 2.0827 - val_loss: 1.1521 - val_mae: 2.0485
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 1.1819 - mae: 2.0823 - val_loss: 1.1434 - val_mae: 2.0340


In [14]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

Unfortunately, when you save the model the threshold will not be saved. You will have to specify the threshold value when loading the model.

In [15]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [16]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.1815 - mean_absolute_error: 2.0822 - val_loss: 1.1431 - val_mean_absolute_error: 2.0434
Epoch 2/2
363/363 [==============================] - 1s 1ms/step - loss: 1.1815 - mean_absolute_error: 2.0822 - val_loss: 1.1526 - val_mean_absolute_error: 2.0452


You can solve this issue, by creating a subclass of the `keras.losses.Loss` class and then implement its get_config() method:

In [30]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true, y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

In [31]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [32]:
model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [33]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.2048 - mae: 2.0888 - val_loss: 1.1472 - val_mae: 2.0366
Epoch 2/2
363/363 [==============================] - 1s 1ms/step - loss: 1.1848 - mae: 2.0826 - val_loss: 1.1512 - val_mae: 2.0449


In [34]:
model.save("my_model_with_a_custom_loss_class.h5")

When you save the model, now the threshold will be saved with it, and when you load the model, you just need to map the class name to the class itself.

In [35]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5", # TODO: check PR #25956
                               custom_objects={"HuberLoss": HuberLoss})

In [36]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.1834 - mean_absolute_error: 2.0821 - val_loss: 1.1555 - val_mean_absolute_error: 2.0440
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 1.1831 - mean_absolute_error: 2.0824 - val_loss: 1.1473 - val_mean_absolute_error: 2.0402


In [37]:
model.loss.threshold

2.0